In [24]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
import plotly.express as px
from sklearn.model_selection import cross_val_score
import sklearn


In [2]:
import pandas as pd
df = pd.read_csv("retouren.csv")

In [3]:
df["user_dob"] = pd.to_datetime(df.user_dob)
df["user_reg_date"] = pd.to_datetime(df.user_reg_date)
df["order_date"] = pd.to_datetime(df.order_date)
df["delivery_date"] = pd.to_datetime(df.delivery_date)

In [4]:
df = df.drop(df[df.item_price == 999].index)
df = df.drop(df[df.delivery_date < df.order_date].index)
df = df.drop(df[df.item_color == "?"].index)

In [5]:
df.loc[df.item_color == "brwon","item_color"] = "brown"
df.loc[df.item_color == "blau","item_color"] = "blue"
df.loc[df.item_color == "dark oliv","item_color"] = "dark olive"

In [6]:
df["age"] = df.order_date.dt.year - df.user_dob.dt.year -((df.order_date.dt.date)<(df.user_dob.dt.date))
df["acc_age"] = df.order_date - df.user_reg_date 
df["acc_age"] = df.acc_age.dt.days

In [7]:
df_mark = df.groupby(["item_id","item_size","item_color"]).agg(maxprice=("item_price","max")).reset_index()
df = pd.merge(df,df_mark)
df["markdown"] = 0
df.loc[df.item_price < df.maxprice,"markdown"] = 1

In [8]:
df.drop(df[df.delivery_date.isna()].index, inplace=True)

In [9]:
px.scatter(df,x="item_id",y="user_id",color="markdown")

In [10]:
#Trainingsdaten
X = df[["item_id","user_id","markdown"]]
y = df["return"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [11]:
X_train

,item_id,user_id,markdown
5061,114,41363,1
52884,1687,40378,0
59665,1552,22100,0
45698,1359,34058,0
63563,1958,40844,0
...,...,...,...
48711,1550,6301,0
24679,328,22407,0
52790,1636,23933,0
49357,1569,20074,0


In [12]:
#Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.
#You can grid search over parameters of all estimators in the pipeline at once.
#You only have to call fit and predict once on your data to fit a whole sequence of estimators.

pipe = make_pipeline(StandardScaler(),
                     SVC(C=5))

In [13]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC(C=5))])

In [14]:
y_predict = pipe.predict(X_test)

In [15]:
y_predict

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [16]:
y_test

65698    0
59855    0
28923    0
37197    0
72805    0
        ..
64716    1
29660    0
48758    0
38099    1
27608    1
Name: return, Length: 13222, dtype: int64

In [17]:
#pipe.score(X_train,y_train,0.2)

In [18]:
acc_score = accuracy_score(y_predict,y_test)
print('accuracy score : ',acc_score)

accuracy score :  0.511722886098926


In [19]:
#cv = ShuffleSplit(n_splits=10, test_size=0.2)

In [22]:
scores = cross_val_score(pipe, X_train, y_train,n_jobs=4,scoring="accuracy")

ValueError: 'accuracy_score' is not a valid scoring value. Use sklearn.metrics.get_scorer_names() to get valid options.

In [25]:
sklearn.metrics.get_scorer_names()
#cross_validate https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_

In [21]:
print(scores)

[0.53214218 0.52410664 0.53195311 0.52878888 0.51801078]
